# llm-serving: Korean Model
* Container: `Data Science 3.0` (studio, python 3.10), `conda_pytorch_p310` (notebook)

### 참조: 
- Model 정보
    - beomi/KoAlpaca-Polyglot-12.8B
        - This model is a fine-tuned version of EleutherAI/polyglot-ko-12.8b on a KoAlpaca Dataset v1.1b
        - https://huggingface.co/beomi/KoAlpaca-Polyglot-12.8B
    - EleutherAI/polyglot-ko-12.8b
        - Polyglot-Ko-12.8B was trained for 167 billion tokens over 301,000 steps on 256 A100 GPUs with the GPT-NeoX framework. It was trained as an autoregressive language model, using cross-entropy loss to maximize the likelihood of predicting the next token.
        - License: Apache 2.0
        - https://huggingface.co/EleutherAI/polyglot-ko-12.8b
    - nlpai-lab/kullm-polyglot-12.8b-v2
        - https://huggingface.co/nlpai-lab/kullm-polyglot-12.8b-v2
        
- Doc
    - Large model inference tutorials
        - https://docs.aws.amazon.com/sagemaker/latest/dg/large-model-inference-tutorials.html
    - Use DJL with the SageMaker Python SDK
        - https://sagemaker.readthedocs.io/en/stable/frameworks/djl/using_djl.html
        
- 블로그
    - https://aws.amazon.com/ko/blogs/machine-learning/deploy-large-models-on-amazon-sagemaker-using-djlserving-and-deepspeed-model-parallel-inference/
    - 코드
        - https://github.com/aws/amazon-sagemaker-examples/blob/main/inference/generativeai/deepspeed/GPT-J-6B_DJLServing_with_PySDK.ipynb

## 0. Install packages and env. setting

In [53]:
import sys

In [54]:
%load_ext autoreload
%autoreload 2
sys.path.append('../utils') # src 폴더 경로 설정

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
install_needed = True  # should only be True once

In [3]:
%%bash

DAEMON_PATH="/etc/docker"
MEMORY_SIZE=10G

FLAG=$(cat $DAEMON_PATH/daemon.json | jq 'has("data-root")')
# echo $FLAG

if [ "$FLAG" == true ]; then
    echo "Already revised"
else
    echo "Add data-root and default-shm-size=$MEMORY_SIZE"
    sudo cp $DAEMON_PATH/daemon.json $DAEMON_PATH/daemon.json.bak
    sudo cat $DAEMON_PATH/daemon.json.bak | jq '. += {"data-root":"/home/ec2-user/SageMaker/.container/docker","default-shm-size":"'$MEMORY_SIZE'"}' | sudo tee $DAEMON_PATH/daemon.json > /dev/null
    sudo service docker restart
    echo "Docker Restart"
fi

Already revised


In [4]:
import sys
import IPython

if install_needed:
    print("installing deps and restarting kernel")
    !{sys.executable} -m pip install -U pip
    !{sys.executable} -m pip install -U sagemaker
    
    IPython.Application.instance().kernel.do_shutdown(True)

installing deps and restarting kernel
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


## 1. SageMaker endpoint 의 추론 도커 이미지 인 DLC image URL 가져오기
- We get DLC image URL for djl-deepspeed 0.21.0 and set SageMaker settings
- Deep learning contatiners
    - https://github.com/aws/deep-learning-containers/blob/master/available_images.md

In [70]:
import sagemaker, boto3
from sagemaker import image_uris

In [71]:
role = sagemaker.get_execution_role()  # execution role for the endpoint
session = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
region = session._region_name
bucket = session.default_bucket()  # bucket to house artifacts
img_uri = image_uris.retrieve(framework="djl-deepspeed", region=region, version="0.22.1")

In [72]:
print (f'IMAGE URI: {img_uri}')

IMAGE URI: 763104351884.dkr.ecr.us-east-1.amazonaws.com/djl-inference:0.22.1-deepspeed0.9.2-cu118


## 2. Set configuration

In [73]:
import os

### 2.1. Model selection
Korean LLMs: Kullm-polyglot-12-8b-v2, Polyglot-Kor-5-8B, KoAlpaca-12-8B

In [74]:
serve_model = "Kullm-polyglot-12-8b-v2"
model_artifact_name = f'./models/{serve_model}.tar.gz' # 모델 패키징 할 파일 이름
serve_model_path = f'./models/{serve_model}'
s3_location = f's3://{bucket}/{serve_model}' # 모델 패키징 S3 위치

In [75]:
print("serve_model: ", serve_model)
print("model_artifact_name: ", model_artifact_name)
print("serve_model_path: ", serve_model_path)
print("model packaging s3 location: ", s3_location)

serve_model:  Kullm-polyglot-12-8b-v2
model_artifact_name:  ./models/Kullm-polyglot-12-8b-v2.tar.gz
serve_model_path:  ./models/Kullm-polyglot-12-8b-v2
model packaging s3 location:  s3://sagemaker-us-east-1-419974056037/Kullm-polyglot-12-8b-v2


### Step 2.2. Create a `model.py` and `serving.properties`

#### 2.2.1. model.py

In [76]:
%%writefile ./models/Kullm-polyglot-12-8b-v2/model.py

import os
import torch
import deepspeed
from djl_python import Input, Output
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, GPTNeoXLayer

predictor = None

def get_model(properties):
    
    model_name = "nlpai-lab/kullm-polyglot-12.8b-v2"
    tensor_parallel = properties["tensor_parallel_degree"]
    local_rank = int(os.getenv("LOCAL_RANK", "0"))
    
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16,
        low_cpu_mem_usage=True,
    )
    
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    
    model.eval()
    model = deepspeed.init_inference(
        model,
        mp_size=tensor_parallel,
        dtype=model.dtype,
        replace_method="auto",
        #replace_with_kernel_inject=True,
        injection_policy={
            GPTNeoXLayer:('attention.dense', 'mlp.dense_4h_to_h')
        }
    )
    generator = pipeline(
        task="text-generation", model=model, tokenizer=tokenizer, device=local_rank
    )
    # https://huggingface.co/docs/hub/models-tasks
    return generator
    
def handle(inputs: Input) -> None:
    
    global predictor
    if not predictor:
        predictor = get_model(inputs.get_properties())

    if inputs.is_empty():
        # Model server makes an empty call to warmup the model on startup
        print ("is_empty")
        return None

    data = inputs.get_as_json() #inputs.get_as_string()
    
    print ("data:",  data)
    
    input_prompt, params = data["prompt"], data["params"]
    
    print ("input_prompt", input_prompt)
    print ("params", params)
    
    result = predictor(
        input_prompt,
        **params
    )
    
    print ("result:", result)
    return Output().add_as_json(result) #Output().add(result)


Overwriting ./models/Kullm-polyglot-12-8b-v2/model.py


#### 2.2.2. serving.properties

In [77]:
%%writefile ./models/Kullm-polyglot-12-8b-v2/serving.properties

engine = DeepSpeed

# passing extra options to model.py or built-in handler
job_queue_size=100
batch_size=1
max_batch_delay=1
max_idle_time=60
#gpu.minWorkers=4
#gpu.maxWorkers=4 

# defines custom environment variables
#env=SERVING_NUMBER_OF_NETTY_THREADS=2

# Allows to load DeepSpeed workers in parallel
option.parallel_loading=true

# specify tensor parallel degree (number of partitions)
option.tensor_parallel_degree=4

# specify per model timeout
option.model_loading_timeout=600
option.predict_timeout=240

# mark the model as failure after python process crashing 10 times
retry_threshold=0

Overwriting ./models/Kullm-polyglot-12-8b-v2/serving.properties


### 2.3. mode selection (local/cloud)
 - **local 모드의 경우 현재 error 발생, 원인 파악중**

In [78]:
use_local_mode = False

if use_local_mode:
    instance_type = "local_gpu"
    from sagemaker.local import LocalSession
    sagemaker_session = LocalSession()
    sagemaker_session.config = {'local': {'local_code': True}}
else:
    sagemaker_session = sagemaker.session.Session()
    instance_type = "ml.g5.12xlarge"

In [79]:
print("instance_type  :", instance_type)

instance_type  : ml.g5.12xlarge


## 3. model packaging
- `model.py` and `serving.properties`
- The code below creates the SageMaker model file (`model.tar.gz`) and upload it to S3. 

In [80]:
%%sh -s {serve_model_path} {model_artifact_name}
serve_model=$1
model_artifact_name=$2
echo $serve_model
echo $model_artifact_name
rm -rf $serve_model_path/.ipynb_checkpoints
tar -czvf $model_artifact_name $serve_model/

./models/Kullm-polyglot-12-8b-v2
./models/Kullm-polyglot-12-8b-v2.tar.gz
./models/Kullm-polyglot-12-8b-v2/
./models/Kullm-polyglot-12-8b-v2/serving.properties
./models/Kullm-polyglot-12-8b-v2/model.py
./models/Kullm-polyglot-12-8b-v2/.ipynb_checkpoints/
./models/Kullm-polyglot-12-8b-v2/.ipynb_checkpoints/model-checkpoint.py
./models/Kullm-polyglot-12-8b-v2/.ipynb_checkpoints/serving-checkpoint.properties


## 4. upload model package

In [81]:
model_tar_url = sagemaker.s3.S3Uploader.upload(model_artifact_name, s3_location)

In [82]:
print("model_tar_url: ", model_tar_url)

model_tar_url:  s3://sagemaker-us-east-1-419974056037/Kullm-polyglot-12-8b-v2/Kullm-polyglot-12-8b-v2.tar.gz


## 5. create sagemaker model

In [83]:
from datetime import datetime
from sagemaker.model import Model
from sagemaker import image_uris, get_execution_role

In [84]:
def create_model(model_name, role, sagemaker_session, inference_image_uri, model_s3_url):
    model = Model(
        image_uri=inference_image_uri,
        model_data=model_s3_url,
        role=role,
        name=model_name,
        sagemaker_session=sagemaker_session,
    )
    return model

In [85]:
time_stamp = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
model_name = f'{serve_model}-' + time_stamp
sm_model = create_model(model_name, role, sagemaker_session, img_uri, model_tar_url)

## 6. create sagemaker endpoint
- It will take about 10 mins

In [86]:
from sagemaker import serializers, deserializers

In [87]:
def deploy_model(model, sagemaker_session, instance_type, _endpoint_name):
    model.deploy(
        initial_instance_count=1,
        instance_type=instance_type,
        endpoint_name=_endpoint_name
    )
    predictor = sagemaker.Predictor(
        endpoint_name=_endpoint_name,
        sagemaker_session=sagemaker_session,
        serializer=serializers.JSONSerializer(),
        deserializer=deserializers.JSONDeserializer()
    )
    return predictor

In [88]:
%%time
endpoint_name = f'{serve_model}-' + time_stamp
predictor = deploy_model(sm_model, sagemaker_session, instance_type, endpoint_name)

----------------------!CPU times: user 138 ms, sys: 10.9 ms, total: 149 ms
Wall time: 11min 34s


In [90]:
print("endpoint_name: ", predictor.endpoint_name)

endpoint_name:  Kullm-polyglot-12-8b-v2-2023-07-10-05-48-59


## 7. Inference

### 7.1 Prompt setting for Kullm 

In [91]:
from inference_utils import Prompter

In [92]:
prompter = Prompter("kullm")

### 7.2 Generator parameters

### options for generation
* **temperature**: Controls randomness in the model. Lower values will make the model more deterministic and higher values will make the model more random. Default value is 1.0.
* **max_new_tokens**: The maximum number of tokens to generate. Default value is 20, max value is 512.
* **repetition_penalty**: Controls the likelihood of repetition, defaults to null.
* **seed**: The seed to use for random generation, default is null.
* **stop**: A list of tokens to stop the generation. The generation will stop when one of the tokens is generated.
* **top_k**: The number of highest probability vocabulary tokens to keep for top-k-filtering. Default value is null, which disables top-k-filtering.
* **top_p**: The cumulative probability of parameter highest probability vocabulary tokens to keep for nucleus sampling, default to null
* **do_sample**: Whether or not to use sampling ; use greedy decoding otherwise. Default value is false.
* **best_of**: Generate best_of sequences and return the one if the highest token logprobs, default to null.
* **details**: Whether or not to return details about the generation. Default value is false.
* **return_full_text**: Whether or not to return the full text or only the generated part. Default value is false.
* **truncate**: Whether or not to truncate the input to the maximum length of the model. Default value is true.
* **typical_p**: The typical probability of a token. Default value is null.
* **watermark**: The watermark to use for the generation. Default value is false.

In [93]:
params = {
    "do_sample":False, 
    "max_new_tokens":128,
    "temperature":0,
    "top_k":0,
    "top_p":0.9,
    "return_full_text":False,
    "repetition_penalty":1.1,
    "presence_penalty":None,
    "eos_token_id":2,
}

### 7.3 QnA

In [94]:
import json
from inference_utils import invoke_inference, parse_response

In [95]:
context = "타워의 높이는 330미터 (1,083피트), [6] 81층 건물과 거의 같은 높이이며 파리에서 가장 높은 구조물입니다.밑면은 정사각형이며 각 변의 크기는 125미터 (410피트) 입니다.건설 기간 동안 에펠탑은 워싱턴 기념탑을 제치고 세계에서 가장 높은 인공 건축물이 되었으며, 1930년 뉴욕 크라이슬러 빌딩이 완공될 때까지 41년 동안 이 타이틀을 유지했습니다.높이 200m와 300m를 모두 뛰어넘은 세계 최초의 구조물이었습니다.1957년 타워 꼭대기에 방송 안테나가 추가되면서 지금은 크라이슬러 빌딩보다 5.2미터 (17피트) 더 높아졌습니다.송신기를 제외하고 에펠탑은 미요 육교 다음으로 프랑스에서 두 번째로 높은 독립형 건축물입니다."
false_context = "타워의 높이는 3미터 (10피트) 이고 [6] 높이는 81층 건물과 거의 같으며 파리에서 가장 높은 구조물입니다.밑면은 정사각형이며 각 변의 크기는 125미터 (410피트) 입니다.건설 기간 동안 에펠탑은 워싱턴 기념탑을 제치고 세계에서 가장 높은 인공 건축물이 되었으며, 1930년 뉴욕 크라이슬러 빌딩이 완공될 때까지 41년 동안 이 타이틀을 유지했습니다.높이 200m와 300m를 모두 뛰어넘은 세계 최초의 구조물이었습니다.1957년 타워 꼭대기에 방송 안테나가 추가되면서 지금은 크라이슬러 빌딩보다 5.2미터 (17피트) 더 높아졌습니다.송신기를 제외하고 에펠탑은 미요 육교 다음으로 프랑스에서 두 번째로 높은 독립형 건축물입니다."
irrelevant_context = "밑면은 정사각형이며 각 변의 크기는 125미터 (410피트) 입니다.건설 기간 동안 에펠탑은 워싱턴 기념탑을 제치고 세계에서 가장 높은 인공 건축물이 되었으며, 1930년 뉴욕 크라이슬러 빌딩이 완공될 때까지 41년 동안 이 타이틀을 유지했습니다.높이 200m와 300m를 모두 뛰어넘은 세계 최초의 구조물이었습니다.1957년 타워 꼭대기에 방송 안테나가 추가되면서 지금은 크라이슬러 빌딩보다 5.2미터 (17피트) 더 높아졌습니다.송신기를 제외하고 에펠탑은 미요 육교 다음으로 프랑스에서 두 번째로 높은 독립형 건축물입니다."

True context

In [96]:
q = "에펠탑의 높이는 얼마입니까?"
c = context
prompt = prompter.generate_prompt(q, c)
data = {
    "prompt": [prompt,],
    "params": params
}
print("prompt: \n", data["prompt"][0])

prompt: 
 아래는 작업을 설명하는 명령어와 추가 컨텍스트를 제공하는 입력이 짝을 이루는 예제입니다. 요청을 적절히 완료하는 응답을 작성하세요.

### 명령어:
에펠탑의 높이는 얼마입니까?

### 입력:
타워의 높이는 330미터 (1,083피트), [6] 81층 건물과 거의 같은 높이이며 파리에서 가장 높은 구조물입니다.밑면은 정사각형이며 각 변의 크기는 125미터 (410피트) 입니다.건설 기간 동안 에펠탑은 워싱턴 기념탑을 제치고 세계에서 가장 높은 인공 건축물이 되었으며, 1930년 뉴욕 크라이슬러 빌딩이 완공될 때까지 41년 동안 이 타이틀을 유지했습니다.높이 200m와 300m를 모두 뛰어넘은 세계 최초의 구조물이었습니다.1957년 타워 꼭대기에 방송 안테나가 추가되면서 지금은 크라이슬러 빌딩보다 5.2미터 (17피트) 더 높아졌습니다.송신기를 제외하고 에펠탑은 미요 육교 다음으로 프랑스에서 두 번째로 높은 독립형 건축물입니다.

### 응답:



In [97]:
%%time 
res = invoke_inference(endpoint_name, data)
parse_response(res)

CPU times: user 20.9 ms, sys: 61 µs, total: 21 ms
Wall time: 2.65 s


'에펠탑의 높이는 330미터(1,083피트)입니다.'

False context

In [62]:
q = "에펠탑의 높이는 얼마입니까?"
c = false_context
prompt = prompter.generate_prompt(q, c)
data = {
    "prompt": [prompt,],
    "params": params
}
print("prompt: \n", data["prompt"][0])

prompt: 
 아래는 작업을 설명하는 명령어와 추가 컨텍스트를 제공하는 입력이 짝을 이루는 예제입니다. 요청을 적절히 완료하는 응답을 작성하세요.

### 명령어:
에펠탑의 높이는 얼마입니까?

### 입력:
타워의 높이는 3미터 (10피트) 이고 [6] 높이는 81층 건물과 거의 같으며 파리에서 가장 높은 구조물입니다.밑면은 정사각형이며 각 변의 크기는 125미터 (410피트) 입니다.건설 기간 동안 에펠탑은 워싱턴 기념탑을 제치고 세계에서 가장 높은 인공 건축물이 되었으며, 1930년 뉴욕 크라이슬러 빌딩이 완공될 때까지 41년 동안 이 타이틀을 유지했습니다.높이 200m와 300m를 모두 뛰어넘은 세계 최초의 구조물이었습니다.1957년 타워 꼭대기에 방송 안테나가 추가되면서 지금은 크라이슬러 빌딩보다 5.2미터 (17피트) 더 높아졌습니다.송신기를 제외하고 에펠탑은 미요 육교 다음으로 프랑스에서 두 번째로 높은 독립형 건축물입니다.

### 응답:



In [63]:
%%time 
res = invoke_inference(endpoint_name, data)
parse_response(res)

CPU times: user 19.9 ms, sys: 0 ns, total: 19.9 ms
Wall time: 993 ms


'에펠탑의 높이는 약 3미터(10피트)입니다.'

Irrelevant context

In [66]:
q = "에펠탑의 높이는 얼마입니까?"
c = irrelevant_context
prompt = prompter.generate_prompt(q, c)
data = {
    "prompt": [prompt,],
    "params": params
}
print("prompt: \n", data["prompt"][0])

prompt: 
 아래는 작업을 설명하는 명령어와 추가 컨텍스트를 제공하는 입력이 짝을 이루는 예제입니다. 요청을 적절히 완료하는 응답을 작성하세요.

### 명령어:
에펠탑의 높이는 얼마입니까?

### 입력:
밑면은 정사각형이며 각 변의 크기는 125미터 (410피트) 입니다.건설 기간 동안 에펠탑은 워싱턴 기념탑을 제치고 세계에서 가장 높은 인공 건축물이 되었으며, 1930년 뉴욕 크라이슬러 빌딩이 완공될 때까지 41년 동안 이 타이틀을 유지했습니다.높이 200m와 300m를 모두 뛰어넘은 세계 최초의 구조물이었습니다.1957년 타워 꼭대기에 방송 안테나가 추가되면서 지금은 크라이슬러 빌딩보다 5.2미터 (17피트) 더 높아졌습니다.송신기를 제외하고 에펠탑은 미요 육교 다음으로 프랑스에서 두 번째로 높은 독립형 건축물입니다.

### 응답:



In [67]:
%%time 
res = invoke_inference(endpoint_name, data)
parse_response(res)

CPU times: user 21 ms, sys: 317 µs, total: 21.4 ms
Wall time: 1.09 s


'에펠탑의 높이는 약 324미터(984피트)입니다.'

## 8. Clean-up

### 8.1. Delete the endpoint
Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [68]:
class clean_up():
    
    def __init__(self, ):    
        pass
    
    def delete_endpoint(self, client, endpoint_name ,is_del_model=True):
        
        response = client.describe_endpoint(EndpointName=endpoint_name)
        EndpointConfigName = response['EndpointConfigName']

        response = client.describe_endpoint_config(EndpointConfigName=EndpointConfigName)
        model_name = response['ProductionVariants'][0]['ModelName']    

        if is_del_model: # 모델도 삭제 여부 임.
            client.delete_model(ModelName=model_name)    

        client.delete_endpoint(EndpointName=endpoint_name)
        client.delete_endpoint_config(EndpointConfigName=EndpointConfigName)    

        print(f'--- Deleted model: {model_name}')
        print(f'--- Deleted endpoint: {endpoint_name}')
        print(f'--- Deleted endpoint_config: {EndpointConfigName}')  

In [69]:
clean = clean_up()
sm_client = boto3.client('sagemaker')

## 2.training 
clean.delete_endpoint(sm_client, endpoint_name ,is_del_model=True)

--- Deleted model: Kullm-polyglot-12-8b-v2-2023-07-10-04-48-35
--- Deleted endpoint: Kullm-polyglot-12-8b-v2-2023-07-10-04-48-35
--- Deleted endpoint_config: Kullm-polyglot-12-8b-v2-2023-07-10-04-48-35
